# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
# To print automl model parameters
from pprint import pprint 
# In order to get model information
from azureml.core import Model # will use to get model information

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProject'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
import pandas as pd

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://raw.githubusercontent.com/Basit040/Capstone-Azure-Machine-Learning/main/titanic.csv"

data = 'https://raw.githubusercontent.com/Basit040/Capstone-Azure-Machine-Learning/main/titanic.csv'
#data1 = TabularDatasetFactory.from_delimited_files(data)
#x,y= clean_data(data1)
#df = pd.concat([x, y], axis = 1)
ds = TabularDatasetFactory.from_delimited_files(data)

In [4]:
x,y =clean_data(ds)

In [5]:
x["Survived"]=y

In [6]:
default_ds=ws.get_default_datastore()

In [7]:
dataSet= TabularDatasetFactory.register_pandas_dataframe(x,target=(default_ds, 'dataAutoML'),name='data AutoML',show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to dataAutoML/cd3356c7-b67c-4f6d-b88a-1d7b9ea67b0e/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [8]:
data=ws.datasets.get('data AutoML')

In [9]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "BasCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

BasCompute exists already

Running


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target=trainCluster,
    training_data=data,
    label_column_name='Survived',
    n_cross_validations=5,
    **automl_settings)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_f08fd127-b3c1-4a88-b255-d846475129f0',
 'target': 'BasCompute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-11T11:04:32.149012Z',
 'endTimeUtc': '2021-02-11T11:37:14.668797Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'BasCompute',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneProject","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-138515","workspace_name":"quick-starts-ws-138515","region":"southcentralus","compute_target":"BasCompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"fe

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: CapstoneProject,
Id: AutoML_f08fd127-b3c1-4a88-b255-d846475129f0_21,
Type: azureml.scriptrun,
Status: Completed)
f1_score_weighted 0.7816946470990944
recall_score_weighted 0.7878378378378379
matthews_correlation 0.510579060785241
recall_score_macro 0.7504105676931764
average_precision_score_macro 0.8222519272490381
accuracy 0.7878378378378379
balanced_accuracy 0.7504105676931764
AUC_weighted 0.8421599173773087
weighted_accuracy 0.8153774194227278
average_precision_score_micro 0.8670139058599029
AUC_macro 0.8421599173773087
average_precision_score_weighted 0.8569878919544502
recall_score_micro 0.7878378378378379
log_loss 0.4923214987844359
AUC_micro 0.8583713217722228
precision_score_macro 0.761486571268355
precision_score_weighted 0.7882371585681727
precision_score_micro 0.7878378378378379
f1_score_macro 0.7479976666531484
f1_score_micro 0.7878378378378378
norm_macro_recall 0.5008211353863528
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_f08fd127-b3c1-4a88-b

In [14]:
#TODO: Save the best model
Best_Model = best_run.register_model(model_path='outputs/model.pkl', model_name='titanic_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(Best_Model)

Model(workspace=Workspace.create(name='quick-starts-ws-138515', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-138515'), name=titanic_automl, id=titanic_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.7878378378378379'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoringScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [16]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# set inference config
inference_config = InferenceConfig(entry_script='scoringScript.py',
                                    environment=best_run.get_environment())


In [17]:
#Set deployment config
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 1)

In [18]:
service = Model.deploy(ws, "basserve", [Best_Model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-138515', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-138515'), name=basserve, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://9712cb3c-ab2e-45a6-bc1d-6e95825bdfd3.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})

In [20]:
# wait for deployment to finish and display the scoring uri and swagger uri
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Healthy
http://9712cb3c-ab2e-45a6-bc1d-6e95825bdfd3.southcentralus.azurecontainer.io/score
http://9712cb3c-ab2e-45a6-bc1d-6e95825bdfd3.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
# Convert data to pandas dataframe
df = data.to_pandas_dataframe().dropna()

In [22]:
import json
test_df = df.sample(5) 
label_df = test_df.pop('Survived')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"PassengerId": 258, "Age": 1, "Fare": 3, "Pclass_2": 0, "Pclass_3": 0, "Fam_type_Small": 0, "Fam_type_Big": 0, "Fam_type_Very big": 0, "Title_Master": 0, "Title_Miss": 1, "Title_Mr": 0, "Title_Mrs": 0, "Embarked_Q": 0, "Embarked_S": 1, "Cabin_B": 1, "Cabin_C": 0, "Cabin_D": 0, "Cabin_E": 0, "Cabin_F": 0, "Cabin_G": 0, "Cabin_T": 0}, {"PassengerId": 292, "Age": 1, "Fare": 3, "Pclass_2": 0, "Pclass_3": 0, "Fam_type_Small": 1, "Fam_type_Big": 0, "Fam_type_Very big": 0, "Title_Master": 0, "Title_Miss": 0, "Title_Mr": 0, "Title_Mrs": 1, "Embarked_Q": 0, "Embarked_S": 0, "Cabin_B": 1, "Cabin_C": 0, "Cabin_D": 0, "Cabin_E": 0, "Cabin_F": 0, "Cabin_G": 0, "Cabin_T": 0}, {"PassengerId": 178, "Age": 3, "Fare": 2, "Pclass_2": 0, "Pclass_3": 0, "Fam_type_Small": 0, "Fam_type_Big": 0, "Fam_type_Very big": 0, "Title_Master": 0, "Title_Miss": 1, "Title_Mr": 0, "Title_Mrs": 0, "Embarked_Q": 0, "Embarked_S": 0, "Cabin_B": 0, "Cabin_C": 1, "Cabin_D": 0, "Cabin_E": 0, "Cabin_F": 0, "Cabin_G": 

TODO: In the cell below, print the logs of the web service and delete the service

In [23]:
output = service.run(test_sample)

In [24]:
print (output)

{"result": [1, 1, 1, 0, 1]}


In [25]:
# Print original labels
print(label_df)

44     1
51     1
29     0
177    0
92     1
Name: Survived, dtype: int64


In [26]:
print(service.get_logs())

2021-02-11T11:52:40,938490282+00:00 - iot-server/run 
2021-02-11T11:52:40,938716684+00:00 - rsyslog/run 
2021-02-11T11:52:40,940719694+00:00 - gunicorn/run 
2021-02-11T11:52:40,947524931+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [27]:
service.delete()